# Week 02

Content:

1. Language modelling using bigrams
2. Necessity of end of sentence token `<\s>`
3. Doing k-smoothing, we are still dealing with a probability
4. Makemore Part I

## 1. Language modelling using bigrams



In [9]:
import nltk
nltk.download("gutenberg")
nltk.download("punkt_tab")
emma = nltk.corpus.gutenberg.raw(fileids='austen-emma.txt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Seya.Schmassmann\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Seya.Schmassmann\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [7]:
print(emma[:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

Split into sentences:

In [10]:
from nltk.tokenize import WhitespaceTokenizer, word_tokenize, sent_tokenize
sentences = sent_tokenize(emma.lower())
sentences[:5]

['[emma by jane austen 1816]\n\nvolume i\n\nchapter i\n\n\nemma woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.',
 "she was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.",
 'her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.',
 "sixteen years had miss taylor been in mr. woodhouse's family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of emma.",
 'between _them_ it was more the intimacy\nof sisters.']

To Do:
- count all bigrams in sentences above (add a start and end of sentence token), since we want to generate new sentences
- construct probabilities from the bigram counter above
- to generate new sentences you start with `<s>` and generate a new word from the bigram probabilities until you reach the end of sentence token `<\s>`
- finally calculate the perplexity of your generated sentences and do the same for original sentences from `sentences`


Tipps:
- `nltk.utils` has a `ngrams` function
- you might want to use `from collections import Counter`
- `np.random.choice(candidate_tokens, size=1, p=candidate_probs)` can be used to draw tokens given computed probabilities

In [34]:
from nltk.util import ngrams
import numpy as np
from collections import Counter

# add start and end tokens
tokenized_sentences = [["<s>"] + word_tokenize(sentence) + ["</s>"] for sentence in sentences]
print(tokenized_sentences[:5])

# count bigrams
bigrams = [tuple(pair) for sentence in tokenized_sentences for pair in ngrams(sentence, 2)]
bigram_counts = Counter(bigrams)
print(bigram_counts.most_common(5))

# compute probabilities
bigram_probabilities = {}
unigram_counts = Counter([word for sentence in tokenized_sentences for word in sentence])
for (w1, w2), count in bigram_counts.items():
    bigram_probabilities[(w1, w2)] = count / unigram_counts.get(w1, 1) # C(w1, w2) / C(w1)
print(bigram_probabilities[(".", "</s>")])


def generate_sentence():
    sentence = ["<s>"]
    while sentence[-1] != "</s>":
        candidates = [w2 for (w1, w2) in bigram_probabilities.keys() if w1 == sentence[-1]]
        probs = [bigram_probabilities[(sentence[-1], w2)] for w2 in candidates]
        if not candidates:
            break
        next_word = np.random.choice(candidates, p=probs)
        sentence.append(next_word)
    return " ".join(sentence[1:-1])


def calculate_perplexity(sentence):
    tokens = ["<s>"] + word_tokenize(sentence) + ["</s>"]
    bigrams = list(nltk.ngrams(tokens, 2))
    prob = 1
    N = len(bigrams)
    for w1, w2 in bigrams:
        prob *= bigram_probabilities.get((w1, w2), 1e-6)
    return prob ** (-1/N)

[['<s>', '[', 'emma', 'by', 'jane', 'austen', '1816', ']', 'volume', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', 'unite', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', 'twenty-one', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', 'vex', 'her', '.', '</s>'], ['<s>', 'she', 'was', 'the', 'youngest', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', 'indulgent', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'s", 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a', 'very', 'early', 'period', '.', '</s>'], ['<s>', 'her', 'mother', 'had', 'died', 'too', 'long', 'ago', 'for', 'her', 'to', 'have', 'more', 'than', 'an', 'indistinct', 'remembrance', 'of', 'her', 'caresses', ';', 'and', 'her', 'pla

Generierung von neuen Sätzen:

In [52]:
generated_sentence = generate_sentence()
print("Generated Sentence:", generated_sentence)
print("Perplexity of Generated Sentence:", calculate_perplexity(generated_sentence))
print("Perplexity of Original Sentences:", calculate_perplexity("Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world with very little to distress or vex her."))

Generated Sentence: `` the whole fortnight ; and was wishing any feeling the sort , you will be summer , there is frank churchill !
Perplexity of Generated Sentence: 35.62228643090355
Perplexity of Original Sentences: 80.28897429408843


## 2. Necessity of `<\s>`

Exercise 3.5 of [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/). Given a trainings corpus without `<\s>`

`<s> a b` <br>
`<s> b a` <br>
`<s> a a` <br>
`<s> b b` <br>

1. Use a bigram model and calculate the probability of all possible sentences with two words $\{a, b\}$. Show that the sum of all these probabilities add up to 1.
2. Do the same for all possible sentences of lengths 3 of the words $\{a, b\}$. Show that these sum up to 1 as well.

The point is, that this property is not very useful: we rather have a language model, that is able to produce a sentence of arbitrary length. When you are generating a sentence, you are not going to decide before hand, how many words you are going to use...

### Answer

First let's write down the formula for bigrams:

$$
\mathbb P [ <\! s \! >, w_1, \dots, w_n] = \mathbb P [w_1 \mid <\!s \! > ] \mathbb P[w_2 \mid w_1] \cdots \mathbb P[w_n \mid w_{n-1}]
$$

here $n=2$ for part 1 and $n=3$ for part 2.

Let's write $\mathbb P[b \mid a] = p_{b|a}$ and use $s$ instead of `<s>`, then from the training corpus we get:


- $p_{a|s} = p_{b|s} = \dots$
- $\dots$

Recall that we are counting the number of occurrences, e.g.: $p_{b|a}= \frac{C(a,b)}{C(a)} = \frac{1}{2}$

#### Answer 2

We must calculate all probabilities of sentences of lengths 3:

- How many are there?

...


## 3. k-smoothing is still a probability

Given a vocabulary $V$ and a bigram language model, applying k-smoothing (or additive smoothing), i.e.

$$\mathbb{P}[w_n \mid w_{n-1}] = \frac{C(w_n, w_{n-1}) + k}{ C(w_{n-1}) + k|V|}$$

we still have a probability.

We have to show, that when summing over all possible $w_n$, the sum still adds up to $1$:

$$
\begin{align*}
    \sum_{w_n \in V} \mathbb P [w_n \mid w_{n-1}] 
        &=\sum_{w_n \in V} \frac{C(w_{n-1}, w_n) + k}{ C(w_{n-1}) + k|V|} \\
        &= \dots = 1
\end{align*}
$$

## 4. Makemore Part I

Watch Part I of [makemore](https://youtu.be/PaCmpygFfXo?si=7PonsCOBoNCcmWHo), until 1:02:56, and code along. We will come back to this video series in later weeks.

- Write a function, that takes a prefix and completes this prefix according to the the implemented bigram-model (e.g. given the prefix `ann` possible completions could be `anna`, `anner`, `ann...`) 
- Loglikelihood:
    - what happens to the loglikelihood if we use large smoothing factor $k$? 
    - Calculate the loglikelihood for all words for different $k$ values.
    - make an appropriate plot with negative loglikelihood on the y-axis and logarithmic $k$ on the x-axis.